In [2]:
import pandas as pd
import geopandas as gpd
import fiona
import matplotlib
import folium

In [3]:
county_shp_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/bdry_counties_in_minnesota/shp_bdry_counties_in_minnesota.zip"

counties_df_2 = gpd.read_file(county_shp_url)

In [4]:
election_cols = [	"State",
	"County ID",
	"Precinct name",
	"Office ID",
	"Office Name", 
	"District", 
	"Candidate Order Code",
	"Candidate Name",
	"Suffix",
	"Incumbent Code",
	"Party Abbreviation",
	"Number of Precincts reporting",
	"Total number of precincts", #Total number of precincts voting for the office
	"Votes for Candidate",
	"Percentage of Votes for Candidate", #Total number of precincts voting for the office
	"Total number of votes for Office in area"]

county_ref_cols = [
	"County ID",
	"County Name",
	"Number of precincts"
]


party_ref_cols = [
	"Party Abbreviation",
	"Party Name",
	"Party ID", 
]


In [5]:
county_all = pd.read_csv("https://electionresultsfiles.sos.state.mn.us/20221108/allracesbycounty.txt", sep = ';', names=election_cols, encoding='latin-1')
party_ref = pd.read_csv("https://electionresultsfiles.sos.state.mn.us/20221108/PartyTbl.txt", sep = ';', names=party_ref_cols)
county_ref = pd.read_csv("https://electionresultsfiles.sos.state.mn.us/20221108/Cntytbl.txt", sep = ';', names=county_ref_cols)

County Governor Data

In [6]:
gov=county_all[county_all['Office Name']=='Governor & Lt Governor']

In [7]:
pivot_gov = pd.pivot_table(gov, values = ["Votes for Candidate","Percentage of Votes for Candidate"], index = ['County ID'], columns = ["Party Abbreviation"])

In [8]:
pivot_gov.columns

MultiIndex([('Percentage of Votes for Candidate', 'DFL'),
            ('Percentage of Votes for Candidate', 'GLC'),
            ('Percentage of Votes for Candidate',  'IA'),
            ('Percentage of Votes for Candidate', 'LMN'),
            ('Percentage of Votes for Candidate',   'R'),
            ('Percentage of Votes for Candidate', 'SWP'),
            ('Percentage of Votes for Candidate',  'WI'),
            (              'Votes for Candidate', 'DFL'),
            (              'Votes for Candidate', 'GLC'),
            (              'Votes for Candidate',  'IA'),
            (              'Votes for Candidate', 'LMN'),
            (              'Votes for Candidate',   'R'),
            (              'Votes for Candidate', 'SWP'),
            (              'Votes for Candidate',  'WI')],
           names=[None, 'Party Abbreviation'])

In [9]:
pivot_gov.columns = ['-'.join(col).strip() for col in pivot_gov.columns.values]

In [10]:
gov_merged = counties_df_2.merge(pivot_gov, left_on = 'COUN', right_on = 'County ID')

In [11]:
gov_merged.to_file('mnGov2022.geojson', driver = 'GeoJSON')

In [12]:
gov_merged.columns

Index(['AREA', 'PERIMETER', 'CTYONLY_', 'CTYONLY_ID', 'COUN', 'CTY_NAME',
       'CTY_ABBR', 'CTY_FIPS', 'MaxSimpTol', 'MinSimpTol', 'Shape_Leng',
       'Shape_Area', 'geometry', 'Percentage of Votes for Candidate-DFL',
       'Percentage of Votes for Candidate-GLC',
       'Percentage of Votes for Candidate-IA',
       'Percentage of Votes for Candidate-LMN',
       'Percentage of Votes for Candidate-R',
       'Percentage of Votes for Candidate-SWP',
       'Percentage of Votes for Candidate-WI', 'Votes for Candidate-DFL',
       'Votes for Candidate-GLC', 'Votes for Candidate-IA',
       'Votes for Candidate-LMN', 'Votes for Candidate-R',
       'Votes for Candidate-SWP', 'Votes for Candidate-WI'],
      dtype='object')

In [30]:
gov_merged["D-R Diff"] = gov_merged["Percentage of Votes for Candidate-DFL"]-gov_merged["Percentage of Votes for Candidate-R"]

In [53]:
gov_merged_map = gov_merged.explore()
folium.Choropleth(gov_merged, data = gov_merged, columns = ['CTY_NAME','D-R Diff'], key_on = 'feature.properties.CTY_NAME',
    fill_color = 'RdYlBu',
    fill_opacity = 0.7,
    line_opacity = .8,
    line_weight = 3.5,
    smooth_factor = 1.5,
    legend_name = 'Vote Share', # Voter Proportion (%) in 2019
    highlight = True,
    bins = 15).add_to(gov_merged_map)
gov_merged_map